In [2]:
## 규원 05.09 중앙일보 클롱링 csv파일 저장 

import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import csv
from tqdm import tqdm  # tqdm 추가
import time

start_date = datetime(2024, 1, 1)
end_date = datetime(2024, 5, 9)


## 날짜 형식을 바꿔주는 함수 
def date_range(start, end):
    for n in range(int((end - start).days) + 1):
        yield start + timedelta(n)

## 중앙일보 url을 넘겨주면, 해당 url에 있는 기사 list를 가져와주는 함수 
def fetch_news_articles(url):
    response = requests.get(url)
    response.raise_for_status()

    soup = BeautifulSoup(response.text, 'html.parser')
    
    articles_url = []
    for article in soup.find_all('li', class_='card'):
        title_link = article.find('a')
        if title_link and title_link.has_attr('href'):
            articles_url.append(title_link['href'])
            
    return articles_url

## 요청받은 url에서 url, 제목, 기사내용으로 변형해서 return 해주는 함수  
def fetch_articles_with_details(url):
    # URL에서 웹 페이지를 요청하고 응답을 받습니다.
    response = requests.get(url)
    response.raise_for_status()  # HTTP 에러가 있을 경우 예외를 발생시킵니다.
    
    # 응답으로부터 HTML을 파싱합니다.
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # 기사 데이터를 저장할 리스트를 초기화합니다.
    articles = []
    
    # 각 기사를 찾아내어 필요한 정보를 추출합니다.
    for article in soup.find_all('article'):
        # 제목 추출
        headline = article.find('h1', class_='headline')
        title = headline.get_text(strip=True) if headline else "No title"
        # 본문 내용 추출
        body = article.find('div', class_='article_body fs3')
        text = []
        if body:
            for p in body.find_all('p', attrs={"data-divno": True}):
                text.append(p.get_text(strip=True))
        text_content = ' '.join(text)
        
        # 기사 URL, 제목, 본문을 튜플로 만들어 리스트에 추가합니다.
        print(url+"진행중")
        articles.append([url, title, text_content])
    
    return articles


# 모든 날짜에 대해 URL을 생성하고 크롤링을 실행, 모든 뉴스 링크 list 전달 
def get_news_links():
    ## 시작 요일부터 하루씩 올려가며, 데이터 크롤링 
    links_url = []
    for single_date in date_range(start_date, end_date):
        formatted_date = single_date.strftime("%Y/%m/%d")
        url = f'https://www.joongang.co.kr/sitemap/index/{formatted_date}'
        links = fetch_news_articles(url)
        print(f"Date: {formatted_date}")
        for link in links:
            links_url.append(link)
    ## 모든 크롤링한 모든 링크의 url 전달 
    return links_url


## 최종적으로 url 제목 본분 행태로 저장해주는 코드 
def joongang_crawler():
    crawled_data = []
    
    ## 모아진 뉴스 링크 가져옴 
    links_list = get_news_links()
    for link in links_list:
        ## 링크에 방문해서 뉴스기사 url 제목 뉴스기사 추출해서 저장
        links_detail = fetch_articles_with_details(link)
        crawled_data.extend(links_detail) # list에 추가,append로 하면 3차원 배열됨  
    #print(crawled_data)
    return crawled_data



def save_to_csv(data):
    # CSV 파일을 열고 데이터를 저장합니다.
    with open('joongang_news.csv', 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        # CSV 파일의 헤더를 작성합니다.
        writer.writerow(['URL', 'Title', 'Content'])
        # 데이터를 CSV 파일에 작성합니다.
        writer.writerows(data)
        
# 데이터가 예시로 준비되어 있다고 가정하고, 이 함수를 사용하여 저장합니다.
# example_data = [
#     ('https://example.com/article1', 'Title 1', 'Content of article 1...'),
#     ('https://example.com/article2', 'Title 2', 'Content of article 2...'),
#     ('https://example.com/article3', 'Title 3', 'Content of article 3...')
# ]

# CSV 파일 저장
if __name__ == "__main__":
    start_time = time.time()  # 코드 실행 시작 시간 기록
    crawled_data = joongang_crawler()
    save_to_csv(crawled_data)
    end_time = time.time()  # 코드 실행 종료 시간 기록
    print(f"Execution time: {end_time - start_time} seconds")  # 실행 시간 출력

Date: 2024/01/01
Date: 2024/01/02
Date: 2024/01/03
Date: 2024/01/04
Date: 2024/01/05
Date: 2024/01/06
Date: 2024/01/07
Date: 2024/01/08
Date: 2024/01/09
Date: 2024/01/10
Date: 2024/01/11
Date: 2024/01/12
Date: 2024/01/13
Date: 2024/01/14
Date: 2024/01/15
Date: 2024/01/16
Date: 2024/01/17
Date: 2024/01/18
Date: 2024/01/19
Date: 2024/01/20
Date: 2024/01/21
Date: 2024/01/22
Date: 2024/01/23
Date: 2024/01/24
Date: 2024/01/25
Date: 2024/01/26
Date: 2024/01/27
Date: 2024/01/28
Date: 2024/01/29
Date: 2024/01/30
Date: 2024/01/31
Date: 2024/02/01
Date: 2024/02/02
Date: 2024/02/03
Date: 2024/02/04
Date: 2024/02/05
Date: 2024/02/06
Date: 2024/02/07
Date: 2024/02/08
Date: 2024/02/09
Date: 2024/02/10
Date: 2024/02/11
Date: 2024/02/12
Date: 2024/02/13
Date: 2024/02/14
Date: 2024/02/15
Date: 2024/02/16
Date: 2024/02/17
Date: 2024/02/18
Date: 2024/02/19
Date: 2024/02/20
Date: 2024/02/21
Date: 2024/02/22
Date: 2024/02/23
Date: 2024/02/24
Date: 2024/02/25
Date: 2024/02/26
Date: 2024/02/27
Date: 2024/02/

https://www.joongang.co.kr/article/25218932진행중
https://www.joongang.co.kr/article/25218933진행중
https://www.joongang.co.kr/article/25218934진행중
https://www.joongang.co.kr/article/25218935진행중
https://www.joongang.co.kr/article/25218936진행중
https://www.joongang.co.kr/article/25218937진행중
https://www.joongang.co.kr/article/25218938진행중
https://www.joongang.co.kr/article/25218939진행중
https://www.joongang.co.kr/article/25218940진행중
https://www.joongang.co.kr/article/25218941진행중
https://www.joongang.co.kr/article/25218942진행중
https://www.joongang.co.kr/article/25218944진행중
https://www.joongang.co.kr/article/25218945진행중
https://www.joongang.co.kr/article/25218946진행중
https://www.joongang.co.kr/article/25218947진행중
https://www.joongang.co.kr/article/25218948진행중
https://www.joongang.co.kr/article/25218949진행중
https://www.joongang.co.kr/article/25218950진행중
https://www.joongang.co.kr/article/25218952진행중
https://www.joongang.co.kr/article/25218953진행중
https://www.joongang.co.kr/article/25218954진행중
https://www.j

https://www.joongang.co.kr/article/25219107진행중
https://www.joongang.co.kr/article/25219108진행중
https://www.joongang.co.kr/article/25219109진행중
https://www.joongang.co.kr/article/25219110진행중
https://www.joongang.co.kr/article/25219111진행중
https://www.joongang.co.kr/article/25219112진행중
https://www.joongang.co.kr/article/25219113진행중
https://www.joongang.co.kr/article/25219114진행중
https://www.joongang.co.kr/article/25219115진행중
https://www.joongang.co.kr/article/25219116진행중
https://www.joongang.co.kr/article/25219117진행중
https://www.joongang.co.kr/article/25219118진행중
https://www.joongang.co.kr/article/25219119진행중
https://www.joongang.co.kr/article/25219120진행중
https://www.joongang.co.kr/article/25219121진행중
https://www.joongang.co.kr/article/25219122진행중
https://www.joongang.co.kr/article/25219123진행중
https://www.joongang.co.kr/article/25219124진행중
https://www.joongang.co.kr/article/25219125진행중
https://www.joongang.co.kr/article/25219126진행중
https://www.joongang.co.kr/article/25219127진행중
https://www.j

https://www.joongang.co.kr/article/25219277진행중
https://www.joongang.co.kr/article/25219278진행중
https://www.joongang.co.kr/article/25219279진행중
https://www.joongang.co.kr/article/25219280진행중
https://www.joongang.co.kr/article/25219281진행중
https://www.joongang.co.kr/article/25219282진행중
https://www.joongang.co.kr/article/25219283진행중
https://www.joongang.co.kr/article/25219284진행중
https://www.joongang.co.kr/article/25219285진행중
https://www.joongang.co.kr/article/25219286진행중
https://www.joongang.co.kr/article/25219287진행중
https://www.joongang.co.kr/article/25219288진행중
https://www.joongang.co.kr/article/25219289진행중
https://www.joongang.co.kr/article/25219290진행중
https://www.joongang.co.kr/article/25219291진행중
https://www.joongang.co.kr/article/25219292진행중
https://www.joongang.co.kr/article/25219293진행중
https://www.joongang.co.kr/article/25219294진행중
https://www.joongang.co.kr/article/25219295진행중
https://www.joongang.co.kr/article/25219296진행중
https://www.joongang.co.kr/article/25219297진행중
https://www.j

https://www.joongang.co.kr/article/25219453진행중
https://www.joongang.co.kr/article/25219454진행중
https://www.joongang.co.kr/article/25219455진행중
https://www.joongang.co.kr/article/25219456진행중
https://www.joongang.co.kr/article/25219457진행중
https://www.joongang.co.kr/article/25219458진행중
https://www.joongang.co.kr/article/25219459진행중
https://www.joongang.co.kr/article/25219460진행중
https://www.joongang.co.kr/article/25219461진행중
https://www.joongang.co.kr/article/25219462진행중
https://www.joongang.co.kr/article/25219463진행중
https://www.joongang.co.kr/article/25219464진행중
https://www.joongang.co.kr/article/25219465진행중
https://www.joongang.co.kr/article/25219466진행중
https://www.joongang.co.kr/article/25219467진행중
https://www.joongang.co.kr/article/25219468진행중
https://www.joongang.co.kr/article/25219469진행중
https://www.joongang.co.kr/article/25219470진행중
https://www.joongang.co.kr/article/25219471진행중
https://www.joongang.co.kr/article/25219472진행중
https://www.joongang.co.kr/article/25219473진행중
https://www.j

...

https://www.joongang.co.kr/article/25247400진행중
https://www.joongang.co.kr/article/25247401진행중
https://www.joongang.co.kr/article/25247402진행중
https://www.joongang.co.kr/article/25247403진행중
https://www.joongang.co.kr/article/25247404진행중
https://www.joongang.co.kr/article/25247405진행중
https://www.joongang.co.kr/article/25247406진행중
https://www.joongang.co.kr/article/25247407진행중
https://www.joongang.co.kr/article/25247408진행중
https://www.joongang.co.kr/article/25247409진행중
https://www.joongang.co.kr/article/25247410진행중
https://www.joongang.co.kr/article/25247411진행중
https://www.joongang.co.kr/article/25247412진행중
https://www.joongang.co.kr/article/25247413진행중
https://www.joongang.co.kr/article/25247414진행중
https://www.joongang.co.kr/article/25247415진행중
https://www.joongang.co.kr/article/25247416진행중
https://www.joongang.co.kr/article/25247417진행중
https://www.joongang.co.kr/article/25247418진행중
https://www.joongang.co.kr/article/25247419진행중
https://www.joongang.co.kr/article/25247420진행중
https://www.j

https://www.joongang.co.kr/article/25247575진행중
https://www.joongang.co.kr/article/25247576진행중
https://www.joongang.co.kr/article/25247577진행중
https://www.joongang.co.kr/article/25247578진행중
https://www.joongang.co.kr/article/25247579진행중
https://www.joongang.co.kr/article/25247580진행중
https://www.joongang.co.kr/article/25247581진행중
https://www.joongang.co.kr/article/25247582진행중
https://www.joongang.co.kr/article/25247583진행중
https://www.joongang.co.kr/article/25247584진행중
https://www.joongang.co.kr/article/25247585진행중
https://www.joongang.co.kr/article/25247586진행중
https://www.joongang.co.kr/article/25247587진행중
https://www.joongang.co.kr/article/25247588진행중
https://www.joongang.co.kr/article/25247589진행중
https://www.joongang.co.kr/article/25247590진행중
https://www.joongang.co.kr/article/25247591진행중
https://www.joongang.co.kr/article/25247592진행중
https://www.joongang.co.kr/article/25247593진행중
https://www.joongang.co.kr/article/25247593진행중
https://www.joongang.co.kr/article/25247593진행중
https://www.j

https://www.joongang.co.kr/article/25247736진행중
https://www.joongang.co.kr/article/25247737진행중
https://www.joongang.co.kr/article/25247738진행중
https://www.joongang.co.kr/article/25247739진행중
https://www.joongang.co.kr/article/25247740진행중
https://www.joongang.co.kr/article/25247741진행중
https://www.joongang.co.kr/article/25247742진행중
https://www.joongang.co.kr/article/25247743진행중
https://www.joongang.co.kr/article/25247744진행중
https://www.joongang.co.kr/article/25247745진행중
https://www.joongang.co.kr/article/25247746진행중
https://www.joongang.co.kr/article/25247747진행중
https://www.joongang.co.kr/article/25247748진행중
https://www.joongang.co.kr/article/25247749진행중
https://www.joongang.co.kr/article/25247750진행중
https://www.joongang.co.kr/article/25247751진행중
https://www.joongang.co.kr/article/25247752진행중
https://www.joongang.co.kr/article/25247753진행중
https://www.joongang.co.kr/article/25247754진행중
https://www.joongang.co.kr/article/25247755진행중
https://www.joongang.co.kr/article/25247756진행중
https://www.j

https://www.joongang.co.kr/article/25247912진행중
https://www.joongang.co.kr/article/25247913진행중
https://www.joongang.co.kr/article/25247914진행중
https://www.joongang.co.kr/article/25247915진행중
https://www.joongang.co.kr/article/25247916진행중
https://www.joongang.co.kr/article/25247917진행중
https://www.joongang.co.kr/article/25247918진행중
https://www.joongang.co.kr/article/25247919진행중
https://www.joongang.co.kr/article/25247920진행중
https://www.joongang.co.kr/article/25247921진행중
https://www.joongang.co.kr/article/25247922진행중
https://www.joongang.co.kr/article/25247923진행중
https://www.joongang.co.kr/article/25247924진행중
https://www.joongang.co.kr/article/25247925진행중
https://www.joongang.co.kr/article/25247926진행중
https://www.joongang.co.kr/article/25247927진행중
https://www.joongang.co.kr/article/25247928진행중
https://www.joongang.co.kr/article/25247929진행중
https://www.joongang.co.kr/article/25247930진행중
https://www.joongang.co.kr/article/25247931진행중
https://www.joongang.co.kr/article/25247932진행중
https://www.j

https://www.joongang.co.kr/article/25248082진행중
https://www.joongang.co.kr/article/25248083진행중
https://www.joongang.co.kr/article/25248084진행중
https://www.joongang.co.kr/article/25248085진행중
https://www.joongang.co.kr/article/25248086진행중
https://www.joongang.co.kr/article/25248087진행중
https://www.joongang.co.kr/article/25248088진행중
https://www.joongang.co.kr/article/25248089진행중
https://www.joongang.co.kr/article/25248090진행중
https://www.joongang.co.kr/article/25248091진행중
https://www.joongang.co.kr/article/25248092진행중
https://www.joongang.co.kr/article/25248093진행중
https://www.joongang.co.kr/article/25248094진행중
https://www.joongang.co.kr/article/25248095진행중
https://www.joongang.co.kr/article/25248096진행중
https://www.joongang.co.kr/article/25248097진행중
https://www.joongang.co.kr/article/25248098진행중
https://www.joongang.co.kr/article/25248099진행중
https://www.joongang.co.kr/article/25248100진행중
https://www.joongang.co.kr/article/25248101진행중
https://www.joongang.co.kr/article/25248102진행중
https://www.j